<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Action Recommendation</b></th>
   </tr>
</table>

<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/action.png" width="550" alt="Icon">

Contents
- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create an Apache Spark machine learning model](#model)
- [4. Store the model in the Watson Machine Learning repository](#persistence)
- [5. Deploy the model in the IBM Cloud](#persistence)
- [6. Score the model](#score)

**Note:** This notebook works correctly with kernel `Python 3.5 with Spark 2.1`.

<a id="setup"></a>
## 0. Setup

In this section please use below cell to upgrade the `watson-machine-learning-client`.

In [1]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client

    100% |████████████████████████████████| 942kB 1.1MB/s eta 0:00:01
Requirement not upgraded as not directly required: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client) (4.23.4)
Requirement not upgraded as not directly required: tabulate in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client) (0.8.2)
Requirement not upgraded as not directly required: urllib3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client) (1.23)
Requirement not upgraded as not directly required: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client) (2018.4.16)
Requirement not upgraded as not directly

**Note**: Please restart the kernel (Kernel -> Restart)

<a id="introduction"></a>
## 1. Introduction

This notebook defines, trains and deploys the model that recommends specific Action for unstatisfied customers.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Read data into Spark DataFrame from DB2 database and show sample record.

### Load data

**TIP:** Put your service credentials here. Just copy paste content of Credentials tab from service details (IBM Cloud)

In [33]:
db2_service_credentials = {
  "username": "***",
  "ssljdbcurl": "***",
  "host": "***",
  "https_url": "***",
  "dsn": "***",
  "hostname": "***",
  "jdbcurl": "***",
  "ssldsn": "***",
  "uri": "***",
  "password": "***"
}

In [34]:
# The code was removed by Watson Studio for sharing.

In [35]:
properties_db2 = {
    'driver': 'com.ibm.db2.jcc.DB2Driver',
    'jdbcurl': db2_service_credentials['jdbcurl'],
    'user': db2_service_credentials['username'],
    'password': db2_service_credentials['password']
}

In [63]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
table_name = 'CAR_RENTAL_TRAINING'
df_data = spark.read.jdbc(properties_db2['jdbcurl'], table='.'.join([properties_db2['user'], table_name]), properties=properties_db2)
df_data.head()

Row(ID=10, Gender='Male', Status='M', Children=1, Age=Decimal('46.000000'), Customer_Status='Inactive', Car_Owner='Yes', Customer_Service='They did not have the car I wanted.  upgraded me to a car I did not like and did not want.', Satisfaction=0, Business_Area='Product: Availability/Variety/Size', Action='Free Upgrade')

### Explore data

In [37]:
df_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Children: integer (nullable = true)
 |-- Age: decimal(14,6) (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- Car_Owner: string (nullable = true)
 |-- Customer_Service: string (nullable = true)
 |-- Satisfaction: integer (nullable = true)
 |-- Business_Area: string (nullable = true)
 |-- Action: string (nullable = true)



**Tip:** Code above can be inserted using Data menu.  You have to select `Insert SparkSession DataFrame` option.

**Note:** Inserted code is modified to work with code in cells below.

As you can see, the data contains eleven fields. `Action` field is the one you would like to predict using feedback data in `Customer_Service` field.

In [38]:
print("Number of records: " + str(df_data.count()))

Number of records: 243


In [39]:
df_data.select('Business_area').groupBy('Business_area').count().show(truncate=False)

+----------------------------------+-----+
|Business_area                     |count|
+----------------------------------+-----+
|Service: Accessibility            |13   |
|Product: Functioning              |75   |
|Service: Attitude                 |12   |
|Service: Orders/Contracts         |16   |
|Product: Availability/Variety/Size|21   |
|Product: Pricing and Billing      |12   |
|Product: Information              |4    |
|Service: Knowledge                |90   |
+----------------------------------+-----+



In [40]:
df_data.select('Action').groupBy('Action').count().show(truncate=False)

+-------------------------+-----+
|Action                   |count|
+-------------------------+-----+
|NA                       |137  |
|Voucher                  |21   |
|Premium features         |15   |
|On-demand pickup location|28   |
|Free Upgrade             |42   |
+-------------------------+-----+



<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data for training a model](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)

<a id="prep"></a>
### 3.1 Prepare data for training a model

In this subsection you will split your data into: train and test data set.

In [41]:
train_data, test_data = df_data.randomSplit([0.8, 0.2], 24)

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 200
Number of testing records : 43


### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In [42]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler, HashingTF, IDF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [43]:
string_indexer_gender = StringIndexer(inputCol="Gender", outputCol="gender_ix")
string_indexer_customer_status = StringIndexer(inputCol="Customer_Status", outputCol="customer_status_ix")
string_indexer_status = StringIndexer(inputCol="Status", outputCol="status_ix")
string_indexer_owner = StringIndexer(inputCol="Car_Owner", outputCol="owner_ix")
string_business_area = StringIndexer(inputCol="Business_Area", outputCol="area_ix")

In [44]:
assembler = VectorAssembler(inputCols=["gender_ix", "customer_status_ix", "status_ix", "owner_ix", "area_ix", "Children", "Age", "Satisfaction"], outputCol="features")

In [45]:
string_indexer_action = StringIndexer(inputCol="Action", outputCol="label").fit(df_data)

In [46]:
label_action_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=string_indexer_action.labels)

In [47]:
dt_action = DecisionTreeClassifier()

In [48]:
pipeline_action = Pipeline(stages=[string_indexer_gender, string_indexer_customer_status, string_indexer_status, string_indexer_action, string_indexer_owner, string_business_area, assembler, dt_action, label_action_converter])

In [49]:
model_action = pipeline_action.fit(train_data)

In [50]:
predictions_action = model_action.transform(test_data)
predictions_action.select('Business_Area','Action','probability','predictedLabel').show(2)

+--------------------+------------+--------------------+--------------+
|       Business_Area|      Action|         probability|predictedLabel|
+--------------------+------------+--------------------+--------------+
|Product: Availabi...|Free Upgrade|[0.0,1.0,0.0,0.0,...|  Free Upgrade|
|Product: Availabi...|Free Upgrade|[0.0,1.0,0.0,0.0,...|  Free Upgrade|
+--------------------+------------+--------------------+--------------+
only showing top 2 rows



In [51]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_action)

print("Accuracy = %g" % accuracy)

Accuracy = 0.767442


<a id="persistence"></a>
## 4. Store the model in the repository

In this section you will store trained model to Watson Machine Learning repository. When model is stored some metada is optional, however we provide it to be able to configure Continuous Learning System.

In [52]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

We need Watson Machine Learning credentials to be able to store model in repository.

**TIP:** Put watson Machine Learning service credentials here.

In [54]:
wml_credentials = {
  "apikey": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "***",
  "iam_serviceid_crn": "***",
  "instance_id": "***",
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}

In [55]:
# The code was removed by Watson Studio for sharing.

In [56]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [57]:
client.version

'1.0.328'

### 4.2 Save the pipeline and model<a id="save"></a>

In [58]:
training_data_reference = {
 "name": "CARS4U training reference",
 "connection": db2_service_credentials,
 "source": {
  "tablename": table_name,
  "type": "dashdb"
 }
}

Define `output_data_schema` for the model

In [69]:
train_data_schema = train_data.schema
label_field = next(f for f in train_data_schema.fields if f.name == "Action")
label_field.metadata['values'] = string_indexer_action.labels

In [72]:
from pyspark.sql.types import *

input_fileds = filter(lambda f: f.name != "Action", train_data_schema.fields)

output_data_schema = StructType(list(input_fileds)). \
    add("prediction", DoubleType(), True, {'modeling_role': 'prediction'}). \
    add("predictedLabel", StringType(), True, {'modeling_role': 'decoded-target', 'values': string_indexer_action.labels}). \
    add("probability", ArrayType(DoubleType()), True, {'modeling_role': 'probability'})

Define model's metadata.

In [75]:
model_props = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Action Recommendation Model",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
    client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: output_data_schema.jsonValue(),
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.7
        }
    ]
}

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available meta names.

Store the model.

In [76]:
published_model_details = client.repository.store_model(model=model_action, meta_props=model_props, training_data=train_data, pipeline=pipeline_action)

In [77]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

cad08901-3a59-47c6-a5a6-08439bcb4bc8


In [81]:
model_details = client.repository.get_model_details(model_uid)

<a id="deploy"></a>
## 5. Deploy model in the IBM Cloud

You can use following command to create online deployment in cloud.

In [78]:
deployment_details = client.deployments.create(model_uid=model_uid, name='CARS4U - Action Model Deployment')



#######################################################################################

Synchronous deployment creation for uid: 'cad08901-3a59-47c6-a5a6-08439bcb4bc8' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='c71edbca-727c-4e7f-9cd7-99929c5b63bf'
------------------------------------------------------------------------------------------------




You can use deployed model to score new data using scoring endpoint.

In [79]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/e30fe554-6e3e-4e0e-af06-90f93686f358/deployments/c71edbca-727c-4e7f-9cd7-99929c5b63bf/online


<a id="score"></a>
## 6. Score the model

In [82]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Business_Area', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0]

In [96]:
payload_scoring = {"fields": fields,"values": [values]}
scoring_response = client.deployments.score(scoring_url, payload_scoring)

print("Recommended action: " + json.dumps(scoring_response['values'][0][21], indent=3))

Recommended action: "Free Upgrade"


---